# Main.py

In [2]:
%tb

import os
import re
import sys
import time
import traceback
from pathlib import Path
import yaml
import click
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException
from src.utils import chrome_browser_options
# from src.llm.llm_manager import GPTAnswerer
from src.authenticator import Authenticator
from src.bot_facade import BotFacade
from src.job_manager import JobManager
# from src.job_application_profile import JobApplicationProfile
from loguru import logger

log_file = "log/app_log.log"
logger.add(log_file)

# Suppress stderr
sys.stderr = open(os.devnull, 'w')

class ConfigError(Exception):
    pass

class ConfigValidator:
    @staticmethod
    def validate_email(email: str) -> bool:
        return re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', email) is not None
    
    
    @staticmethod
    def validate_phone(phone_number: str) -> bool:
        # Delete all parenthesis and '-' symbols from number
        cleaned_number = re.sub(r'[()\-]', '', phone_number)
        
        # Regular expression to match a valid phone number
        pattern = r"^\+?[1-9][0-9]{7,14}$"
        
        # Match the phone number with the pattern
        return re.match(pattern, cleaned_number) is not None

    
    @staticmethod
    def validate_yaml_file(yaml_path: Path) -> dict:
        try:
            with open(yaml_path, 'r') as stream:
                return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            raise ConfigError(f"Error reading file {yaml_path}: {exc}")
        except FileNotFoundError:
            raise ConfigError(f"File not found: {yaml_path}")
    
    
    def validate_config(self, config_yaml_path: Path) -> dict:
        parameters = self.validate_yaml_file(config_yaml_path)
        required_keys = {
            'login': str,
            'keywords' : list,
            'search_only': dict,
            'words_to_exclude' : list,
            'specialization': str,
            'industry': str,
            'regions' : list,
            'districts' : list,
            'subway' : list,
            'income': int,
            'education': dict,
            'experience': dict,
            'job_type': dict,
            'work_schedule': dict,
            'side_job': dict,
            'other_params': dict,
            'llm_model_type': str,
            'llm_model': str
        }
        
        # Проверить что все ключи находятся в файле настроек, а их поля имеют ожидаемый тип
        for key, expected_type in required_keys.items():
            if key not in parameters:
                    raise ConfigError(f"Отсутствует или неверный тип ключа '{key}' в конфигурационном файле {config_yaml_path}")
            elif not isinstance(parameters[key], expected_type):
                raise ConfigError(f"Неверный тип ключа '{key}' в конфигурационном файле {config_yaml_path}. Ожидается {expected_type}.")

        # Проверить все поля и значения настройки "Искать только"
        search_only_list = ['vacancy_name', 'company_name', 'vacancy_description']
        for search in search_only_list:
            if not isinstance(parameters['search_only'].get(search), bool):
                raise ConfigError(f"Поле 'search_only -> {search}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
            
        # Проверить все поля и значения настройки "Образование"
        education = ['not_needed', 'middle', 'higher']
        for edu in education:
            if not isinstance(parameters['education'].get(edu), bool):
                raise ConfigError(f"Поле 'education -> {edu}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Опыт"
        experience = ['doesnt_matter', 'no_experience', 'between_1_and_3', 'between_3_and_6', '6_and_more']
        exp_value_counter = 0
        for exp in experience:
            exp_value = parameters['experience'].get(exp)
            exp_value_counter += exp_value
            if not isinstance(exp_value, bool):
                raise ConfigError(f"Поле 'experience -> {exp}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
        if exp_value_counter > 1:
            raise ConfigError(f"Среди значение 'experience' только одно может иметь значение true в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Тип занятости"
        job_type = ['full_time', 'part_time', 'project', 'volunteer', 'probation', 'civil_law_contract']
        for j_t in job_type:
            if not isinstance(parameters['job_type'].get(j_t), bool):
                raise ConfigError(f"Поле 'job_type -> {j_t}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "График работы"
        work_schedule = ['full_day', 'shift', 'flexible', 'remote', 'fly_in_fly_out']
        for w_s in work_schedule:
            if not isinstance(parameters['work_schedule'].get(w_s), bool):
                raise ConfigError(f"Поле 'work_schedule -> {w_s}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
            
        # Проверить все поля и значения настройки "Подработка"
        side_job = ['project', 'part', 'from_4_hours_per_day', 'weekend', 'evenings']
        for s_j in side_job:
            if not isinstance(parameters['side_job'].get(s_j), bool):
                raise ConfigError(f"Поле 'side_job -> {s_j}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
            
        # Проверить все поля и значения настройки "Другие параметры"
        other_params = ['with_address', 'accept_handicapped', 'not_from_agency', 'accept_kids', 'accredited_it', 'low_performance']
        for o_p in other_params:
            if not isinstance(parameters['other_params'].get(o_p), bool):
                raise ConfigError(f"Поле 'other_params -> {o_p}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Сортировка"
        sort_by = ['relevance', 'publication_time', 'salary_desc', 'salary_asc']
        sort_value_counter = 0
        for s_b in sort_by:
            sort_value = parameters['sort_by'].get(s_b)
            sort_value_counter += sort_value
            if not isinstance(sort_value, bool):
                raise ConfigError(f"Поле 'sort_by -> {s_b}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
        if sort_value_counter > 1:
            raise ConfigError(f"Среди значение 'sort_by' только одно может иметь значение true в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Выводить"
        output_period = ['all_time', 'month', 'week', 'three_days', 'one_day']
        output_value_counter = 0
        for o_p in output_period:
            output_period_value = parameters['output_period'].get(o_p)
            output_value_counter += output_period_value
            if not isinstance(output_period_value, bool):
                raise ConfigError(f"Поле 'output_value -> {o_p}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
        if output_value_counter > 1:
            raise ConfigError(f"Среди значение 'output_period' только одно может иметь значение true в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Показывать на странице"
        output_size = ['show_20', 'show_50', 'show_100']
        output_size_value_counter = 0
        for o_s in output_size:
            output_size_value = parameters['output_size'].get(o_s)
            output_size_value_counter += output_size_value
            if not isinstance(output_size_value, bool):
                raise ConfigError(f"Поле 'output_size -> {o_s}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
        if output_size_value_counter > 1:
            raise ConfigError(f"Среди значение 'output_size' только одно может иметь значение true в конфигурационном файле {config_yaml_path}")
        
        return parameters

    @staticmethod
    def validate_secrets(secrets_yaml_path: Path) -> tuple:
        secrets = ConfigValidator.validate_yaml_file(secrets_yaml_path)
        mandatory_secrets = ['llm_api_key']

        for secret in mandatory_secrets:
            if secret not in secrets:
                raise ConfigError(f"Missing secret '{secret}' in file {secrets_yaml_path}")

        if not secrets['llm_api_key']:
            raise ConfigError(f"llm_api_key cannot be empty in secrets file {secrets_yaml_path}.")
        return secrets['llm_api_key']


class FileManager:
    @staticmethod
    def find_file(name_containing: str, with_extension: str, at_path: Path) -> Path:
        return next((file for file in at_path.iterdir() if name_containing.lower() in file.name.lower() and file.suffix.lower() == with_extension.lower()), None)

    @staticmethod
    def validate_data_folder(app_data_folder: Path) -> tuple:
        """Проверяет, что все необходимые файлы находятся в папке данных"""
        if not app_data_folder.exists() or not app_data_folder.is_dir():
            raise FileNotFoundError(f"Data folder not found: {app_data_folder}")

        required_files = ['secrets.yaml', 'config.yaml', 'plain_text_resume.yaml']
        missing_files = [file for file in required_files if not (app_data_folder / file).exists()]
        
        if missing_files:
            raise FileNotFoundError(f"В папке данных отсутствуют файлы: {', '.join(missing_files)}")

        output_folder = app_data_folder / 'output'
        output_folder.mkdir(exist_ok=True)
        return (app_data_folder / 'secrets.yaml', app_data_folder / 'config.yaml', app_data_folder / 'plain_text_resume.yaml', output_folder)

    @staticmethod
    def file_paths_to_dict(resume_file: Path | None, plain_text_resume_file: Path) -> dict:
        if not plain_text_resume_file.exists():
            raise FileNotFoundError(f"Plain text resume file not found: {plain_text_resume_file}")

        result = {'plainTextResume': plain_text_resume_file}

        if resume_file:
            if not resume_file.exists():
                raise FileNotFoundError(f"Resume file not found: {resume_file}")
            result['resume'] = resume_file

        return result


def init_driver() -> webdriver.Chrome:
    try:
        options = chrome_browser_options()
        service = ChromeService(ChromeDriverManager().install())
        return webdriver.Chrome(service=service, options=options)
    except Exception as e:
        raise RuntimeError(f"Failed to initialize browser: {str(e)}")


def create_and_run_bot(parameters, llm_api_key):
    try:
        driver = init_driver()
        login_component = Authenticator(driver)
        apply_component = JobManager(driver)
        # gpt_answerer_component = GPTAnswerer(parameters, llm_api_key)
        bot = BotFacade(login_component, apply_component)
        # bot.set_job_application_profile_and_resume(job_application_profile_object, resume_object)
        # bot.set_gpt_answerer_and_resume_generator(gpt_answerer_component, resume_generator_manager)
        bot.set_parameters(parameters)
        bot.start_login()
        bot.set_search_parameters()
        # bot.start_apply()
    except WebDriverException as e:
        logger.error(f"WebDriver error occurred: {e}")
    # except Exception as e:
    #     raise RuntimeError(f"Error running the bot: {str(e)}")

def get_traceback(exc: Exception) -> str:
    tb = traceback.extract_tb(exc.__traceback__)[-1]
    return f"line {tb.lineno} in file {tb.filename}"

# @click.command()
# @click.option('--resume', type=click.Path(exists=True, file_okay=True, dir_okay=False, path_type=Path), help="Path to the resume PDF file")
def main(resume: Path = None):
    try:
        data_folder = Path("data_folder")
        secrets_file, config_file, plain_text_resume_file, output_folder = FileManager.validate_data_folder(data_folder)
        
        config_validator = ConfigValidator()
        parameters = config_validator.validate_config(config_file)
        llm_api_key = config_validator.validate_secrets(secrets_file)
        
        # parameters['uploads'] = FileManager.file_paths_to_dict(resume, plain_text_resume_file)
        # parameters['outputFileDirectory'] = output_folder
        
        create_and_run_bot(parameters, llm_api_key)
    except ConfigError as ce:
        logger.error(f"Configuration error: {str(ce)}")
        logger.error(f"Refer to the configuration guide for troubleshooting: https://github.com/feder-cr/AIHawk_AIHawk_automatic_job_application/blob/main/readme.md#configuration {str(ce)}")
    except FileNotFoundError as fnf:
        logger.error(f"File not found: {str(fnf)}")
        logger.error("Ensure all required files are present in the data folder.")
        logger.error("Refer to the file setup guide: https://github.com/feder-cr/AIHawk_AIHawk_automatic_job_application/blob/main/readme.md#configuration")
    except RuntimeError as re:
        tb = get_traceback(re)
        logger.error(f"Runtime error: {str(re)}, {tb}")
        logger.error("Refer to the configuration and troubleshooting guide: https://github.com/feder-cr/AIHawk_AIHawk_automatic_job_application/blob/main/readme.md#configuration")
    except Exception as e:
        tb = get_traceback(e)
        logger.error(f"An unexpected error occurred: {str(e)}, {tb}")
        logger.error("Refer to the general troubleshooting guide: https://github.com/feder-cr/AIHawk_AIHawk_automatic_job_application/blob/main/readme.md#configuration")

# main()

driver = init_driver()
driver.get("https://hh.ru/search/vacancy/advanced")

WebDriverException: Message: unknown error: cannot determine loading status
from target frame detached
  (Session info: chrome=129.0.6668.100)
Stacktrace:
#0 0x5cf5cf6adb9a <unknown>
#1 0x5cf5cf3934cd <unknown>
#2 0x5cf5cf37b442 <unknown>
#3 0x5cf5cf379503 <unknown>
#4 0x5cf5cf379cbf <unknown>
#5 0x5cf5cf389be0 <unknown>
#6 0x5cf5cf39fa04 <unknown>
#7 0x5cf5cf3a4e5b <unknown>
#8 0x5cf5cf37a3ce <unknown>
#9 0x5cf5cf39f551 <unknown>
#10 0x5cf5cf42703b <unknown>
#11 0x5cf5cf407923 <unknown>
#12 0x5cf5cf3d56e7 <unknown>
#13 0x5cf5cf3d66de <unknown>
#14 0x5cf5cf67766b <unknown>
#15 0x5cf5cf67b611 <unknown>
#16 0x5cf5cf6634e5 <unknown>
#17 0x5cf5cf67c192 <unknown>
#18 0x5cf5cf6486ef <unknown>
#19 0x5cf5cf69c9d8 <unknown>
#20 0x5cf5cf69cba7 <unknown>
#21 0x5cf5cf6ac9ec <unknown>
#22 0x7dfe70294ac3 <unknown>


In [4]:
driver.find_element("xpath", f"//*[@data-qa='vacancy-response-link-top']")

<selenium.webdriver.remote.webelement.WebElement (session="c0bde9859244b67322fc5fc4a33de215", element="f.CA647680189F9E66C2DF0F61B8D31F69.d.C23E4F95A46DDF6028D183ECF1CD90D0.e.247")>

# Advanced search

In [66]:
def select_suggestion(driver: webdriver.Chrome, wait: WebDriverWait) -> None:
    """
    Функция для поиска и выбора первой подсказки в списке подсказок, 
    появляющихся во время поиска
    """
    suggest_item = ("class name", "suggest__item")
    wait.until(EC.element_to_be_clickable(suggest_item))
    suggestion_list = driver.find_elements(*suggest_item)
    if len(suggestion_list) > 0:
        suggestion_list[0].click()


def find_by_text_and_click(text: str, driver: webdriver.Chrome) -> None:
    """Функция для поиска элемента по тексту и клика по нему"""
    element = driver.find_element("xpath", f'//*[text()="{text}"]')
    driver.execute_script("arguments[0].scrollIntoView();", element)
    element.click()


def find_by_data_qa_and_click(text: str, driver: webdriver.Chrome) -> None:
    """Функция для поиска элемента по тексту и клика по нему"""
    element = driver.find_element("css selector", f'[data-qa="{text}"]')
    driver.execute_script("arguments[0].scrollIntoView();", element)
    element.click()


def select_checkbox(options: List[str], driver: webdriver.Chrome, type="data_qa") -> None:
    """Функция для выбора всех нужных опций в checkbox"""
    for option in options:
        if type == "data_qa":
            find_by_data_qa_and_click(option, driver)
        else:
            find_by_text_and_click(option, driver)

In [69]:
driver.get("https://hh.ru/search/vacancy/advanced")
wait = WebDriverWait(driver, 15, poll_frequency=1)

# Ключевые слова
keywords = "Data scientist"
keywords_element = driver.find_element("css selector", '[data-qa="vacancysearch__keywords-input"]')
keywords_element.send_keys(keywords)
select_suggestion(driver, wait)


# Искать только
search_only = ["vacancy_name"]
search_only_dict = {
    "vacancy_name": "в названии вакансии", 
    "company_name": "в названии компании", 
    "vacancy_description": "в описании вакансии" 
    }
for s_o in search_only:
    find_by_text_and_click(search_only_dict[s_o], driver)


# Исключить слова
words_to_exclude = "Analyst"
words_to_exclude_element = driver.find_element("css selector", '[data-qa="vacancysearch__keywords-excluded-input"]')
words_to_exclude_element.send_keys(words_to_exclude)


# Cпециализация
# Выбрать меню 'Указать специализации'
specialization = "Программист"
find_by_data_qa_and_click("resumesearch__profroles-switcher", driver)
specialization_item = ("css selector", '[data-qa="bloko-tree-selector-popup-search"]')
# Ввести специализацию в поисковую строку
wait.until(EC.visibility_of_element_located(specialization_item))
specialization_element = driver.find_element("css selector", '[data-qa="bloko-tree-selector-popup-search"]')
specialization_element.send_keys(specialization)
pause()
# Попытаться найти специализацию с таким же названием, что и название специализации в переменной
specialization_list = driver.find_elements("xpath", f'//*[text()="{specialization}"]')
# Иначе выбрать первую специализацию в чекбоксе (если есть, иначе закрыть)
if len(specialization_list) == 0:
    specialization_list = driver.find_elements("css selector", '[data-qa^="bloko-tree-selector-item-text bloko-tree-selector-item-text"]')
if len(specialization_list) > 0:
    driver.execute_script("arguments[0].scrollIntoView();", specialization_list[0])
    specialization_list[0].click()
    driver.find_element("css selector", '[data-qa="bloko-tree-selector-popup-submit"]').click()
else:
    driver.find_element("css selector", '[data-qa="bloko-modal-close"]').click()

# Отрасль компании
# Выбрать меню 'Указать отрасль компании'
industry = "Банк"
find_by_data_qa_and_click("industry-addFromList", driver)
industry_item = ("css selector", '[data-qa="bloko-tree-selector-popup-search"]')
# Ввести отрасль в поисковую строку
wait.until(EC.visibility_of_element_located(industry_item))
industry_element = driver.find_element(*industry_item)
industry_element.clear()
industry_element.send_keys(industry)
pause()
# Попытаться найти отрасль с таким же названием, что и название отрасли в переменной
industry_list = driver.find_elements("xpath", f'//*[text()="{industry}"]')
# Иначе выбрать первую отрасль в чекбоксе (если есть, иначе закрыть)
if len(industry_list) == 0:
    industry_list = driver.find_elements("css selector", '[data-qa^="bloko-tree-selector-item-text bloko-tree-selector-item-text"]')
if len(industry_list) > 0:
    driver.execute_script("arguments[0].scrollIntoView();", industry_list[0])
    industry_list[0].click()
    driver.find_element("css selector", '[data-qa="bloko-tree-selector-popup-submit"]').click()
else:
    driver.find_element("css selector", '[data-qa="bloko-modal-close"]').click()


# Регион
regions = ["Москва"]
region_element = driver.find_element("css selector", '[data-qa="advanced-search-region-add"]')
for region in regions:
    region_element.send_keys(region)
    pause()
    select_suggestion(driver, wait)


# Район (если есть)
districts = ["Замоскворечье", "Нагатино"]
try:
    district_element = driver.find_element("css selector", '[data-qa="searchform__district-input"]')
except NoSuchElementException:
    pass
else:
    for district in districts:
        district_element.send_keys(district)
        pause()
        select_suggestion(driver, wait)
        

# Метро (если есть)
subway = ["Курская", "Таганская"]
try:
    subway_element = driver.find_element("css selector", '[data-qa="searchform__subway-input"]')
except NoSuchElementException:
    pass
else:
    for station in subway:
        subway_element.clear()
        subway_element.send_keys(station)
        pause()
        select_suggestion(driver, wait)


# Уровень дохода
income = 300000
income_element = driver.find_element("css selector", '[data-qa="advanced-search-salary"]')
income_element.clear()
income_element.send_keys(income)


# Образование
education = ["higher"]
education_dict = {
    "not_needed": "advanced-search__education-item-label_not_required_or_not_specified",
    "middle": "advanced-search__education-item-label_special_secondary",
    "higher": "advanced-search__education-item-label_higher",
}
for edu in education:
    find_by_data_qa_and_click(education_dict[edu], driver)


# Требуемый опыт работы
experience = "between_1_and_3"
experience_dict = {
    "doesnt_matter": "advanced-search__experience-item-label_doesNotMatter",
    "no_experience": "advanced-search__experience-item-label_noExperience",
    "between_1_and_3": "advanced-search__experience-item-label_between1And3",
    "between_3_and_6": "advanced-search__experience-item-label_between3And6",
    "more_than_6": "advanced-search__experience-item-label_moreThan6",
    }
find_by_data_qa_and_click(experience_dict[experience], driver)


# Тип занятости
job_type = ["full_time"]
job_type_dict = {
    "full_time": "advanced-search__employment-item-label_full",
    "part_time": "advanced-search__employment-item-label_part",
    "project": "advanced-search__employment-item-label_project",
    "volunteering": "advanced-search__employment-item-label_volunteer",
    "internship": "advanced-search__employment-item-label_probation",
    "civil_law_contract": "Оформление по ГПХ или по совместительству",
}
for j_t in job_type:
    if j_t == "civil_law_contract":
        find_by_text_and_click(job_type_dict[j_t], driver)
    else:
        find_by_data_qa_and_click(job_type_dict[j_t], driver)


# График работы
work_schedule = ["full_day"]
work_schedule_dict = {
    "full_day": "advanced-search__schedule-item-label_fullDay",
    "shift": "advanced-search__schedule-item-label_shift",
    "flexible": "advanced-search__schedule-item-label_flexible",
    "remote": "advanced-search__schedule-item-label_remote",
    "fly_in_fly_out": "advanced-search__schedule-item-label_flyInFlyOut",

}
for w_s in work_schedule:
    find_by_data_qa_and_click(work_schedule_dict[w_s], driver)


# Подработка
side_job = ["weekend"]
side_job_dict = {
    "project": "advanced-search__part_time-item-label_employment_project",
    "part": "advanced-search__part_time-item-label_employment_part" ,
    "from_4_hours_per_day": "advanced-search__part_time-item-label_from_four_to_six_hours_in_a_day",
    "weekend": "advanced-search__part_time-item-label_only_saturday_and_sunday",
    "evenings": "advanced-search__part_time-item-label_start_after_sixteen",
}
for s_j in side_job:
    find_by_data_qa_and_click(side_job_dict[s_j], driver)


# Другие параметры
other_params = ["with_address"]
other_params_dict = {
    "with_address": "advanced-search__label-item-label_with_address", 
    "accept_handicapped": "advanced-search__label-item-label_accept_handicapped",
    "not_from_agency": "advanced-search__label-item-label_not_from_agency",
    "accept_kids": "advanced-search__label-item-label_accept_kids",
    "accredited_it": "advanced-search__label-item-label_accredited_it",
    "low_performance": "advanced-search__label-item-label_low_performance",
    }
for o_p in other_params:
    find_by_data_qa_and_click(other_params_dict[o_p], driver)


# Сортировка
sort_by = "relevance"
sort_by_dict = {
    "relevance": "advanced-search__order_by-item-label_relevance",
    "publication_time": "advanced-search__order_by-item-label_publication_time",
    "salary_desc": "advanced-search__order_by-item-label_salary_desc",
    "salary_asc": "advanced-search__order_by-item-label_salary_asc",
    }
find_by_data_qa_and_click(sort_by_dict[sort_by], driver)


# Выводить за
output_period = "all_time"
output_period_dict = {
    "all_time": "advanced-search__search_period-item-label_0",
    "month": "advanced-search__search_period-item-label_30",
    "week": "advanced-search__search_period-item-label_7",
    "three_days": "advanced-search__search_period-item-label_3",
    "one_day": "advanced-search__search_period-item-label_1",
    }
find_by_data_qa_and_click(output_period_dict[output_period], driver)


# Показывать на странице
output_size = "show_50"
output_size_dict = {
    "show_20": "advanced-search__items_on_page-item-label_20",
    "show_50": "advanced-search__items_on_page-item-label_50",
    "show_100": "advanced-search__items_on_page-item-label_100",
    }
find_by_data_qa_and_click(output_size_dict[output_size], driver)

# main.py

### validate data folder

In [71]:
from typing import List, Dict

from pathlib import Path
import re
import yaml

def validate_data_folder(app_data_folder: Path) -> tuple:
    """Проверяет, что все необходимые файлы находятся в папке данных"""
    if not app_data_folder.exists() or not app_data_folder.is_dir():
        raise FileNotFoundError(f"Data folder not found: {app_data_folder}")

    required_files = ['secrets.yaml', 'config.yaml', 'plain_text_resume.yaml']
    missing_files = [file for file in required_files if not (app_data_folder / file).exists()]
    
    if missing_files:
        raise FileNotFoundError(f"В папке данных отсутствуют файлы: {', '.join(missing_files)}")

    output_folder = app_data_folder / 'output'
    output_folder.mkdir(exist_ok=True)
    return (app_data_folder / 'secrets.yaml', app_data_folder / 'config.yaml', app_data_folder / 'plain_text_resume.yaml', output_folder)

    
data_folder = Path("data_folder")
secrets_file, config_file, plain_text_resume_file, output_folder = validate_data_folder(data_folder)

In [72]:
class ConfigError(Exception):
    pass

class ConfigValidator:
    @staticmethod
    def validate_email(email: str) -> bool:
        return re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', email) is not None
    
    
    @staticmethod
    def validate_phone(phone_number: str) -> bool:
        # Delete all parenthesis and '-' symbols from number
        cleaned_number = re.sub(r'[()\-]', '', phone_number)
        
        # Regular expression to match a valid phone number
        pattern = r"^\+?[1-9][0-9]{7,14}$"
        
        # Match the phone number with the pattern
        return re.match(pattern, cleaned_number) is not None

    
    @staticmethod
    def validate_yaml_file(yaml_path: Path) -> dict:
        try:
            with open(yaml_path, 'r') as stream:
                return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            raise ConfigError(f"Error reading file {yaml_path}: {exc}")
        except FileNotFoundError:
            raise ConfigError(f"File not found: {yaml_path}")
    
    
    def validate_config(self, config_yaml_path: Path) -> dict:
        parameters = self.validate_yaml_file(config_yaml_path)
        required_keys = {
            'keywords' : list,
            'search_only': dict,
            'words_to_exclude' : list,
            'specialization': str,
            'industry': str,
            'region' : list,
            'district' : list,
            'subway' : list,
            'income': int,
            'education': dict,
            'experience': dict,
            'job_type': dict,
            'work_schedule': dict,
            'side_job': dict,
            'other_params': dict,
            'llm_model_type': str,
            'llm_model': str
        }
        
        # Проверить что все ключи находятся в файле настроек, а их поля имеют ожидаемый тип
        for key, expected_type in required_keys.items():
            if key not in parameters:
                    raise ConfigError(f"Отсутствует или неверный тип ключа '{key}' в конфигурационном файле {config_yaml_path}")
            elif not isinstance(parameters[key], expected_type):
                raise ConfigError(f"Неверный тип ключа '{key}' в конфигурационном файле {config_yaml_path}. Ожидается {expected_type}.")

        # Проверить все поля и значения настройки "Искать только"
        search_only_list = ['vacancy_name', 'company_name', 'vacancy_description']
        for search in search_only_list:
            if not isinstance(parameters['search_only'].get(search), bool):
                raise ConfigError(f"Поле 'search_only -> {search}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
            
        # Проверить все поля и значения настройки "Образование"
        education = ['not_needed', 'middle', 'higher']
        for edu in education:
            if not isinstance(parameters['education'].get(edu), bool):
                raise ConfigError(f"Поле 'education -> {edu}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Опыт"
        experience = ['doesnt_matter', 'no_experience', 'between_1_and_3', 'between_3_and_6', '6_and_more']
        exp_value_counter = 0
        for exp in experience:
            exp_value = parameters['experience'].get(exp)
            exp_value_counter += exp_value
            if not isinstance(exp_value, bool):
                raise ConfigError(f"Поле 'experience -> {exp}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
        if exp_value_counter > 1:
            raise ConfigError(f"Среди значение 'experience' только одно может иметь значение true в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Тип занятости"
        job_type = ['full_time', 'part_time', 'project', 'volunteer', 'probation', 'civil_law_contract']
        for j_t in job_type:
            if not isinstance(parameters['job_type'].get(j_t), bool):
                raise ConfigError(f"Поле 'job_type -> {j_t}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "График работы"
        work_schedule = ['full_day', 'shift', 'flexible', 'remote', 'fly_in_fly_out']
        for w_s in work_schedule:
            if not isinstance(parameters['work_schedule'].get(w_s), bool):
                raise ConfigError(f"Поле 'work_schedule -> {w_s}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
            
        # Проверить все поля и значения настройки "Подработка"
        side_job = ['one_time_task', 'part', 'from_4_hours_per_day', 'weekend', 'evenings']
        for s_j in side_job:
            if not isinstance(parameters['side_job'].get(s_j), bool):
                raise ConfigError(f"Поле 'side_job -> {s_j}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
            
        # Проверить все поля и значения настройки "Другие параметры"
        other_params = ['with_address', 'accept_handicapped', 'not_from_agency', 'accept_kids', 'accredited_it', 'low_performance']
        for o_p in other_params:
            if not isinstance(parameters['other_params'].get(o_p), bool):
                raise ConfigError(f"Поле 'other_params -> {o_p}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Сортировка"
        sort_by = ['relevance', 'publication_time', 'salary_desc', 'salary_asc']
        sort_value_counter = 0
        for s_b in sort_by:
            sort_value = parameters['sort_by'].get(s_b)
            sort_value_counter += sort_value
            if not isinstance(sort_value, bool):
                raise ConfigError(f"Поле 'sort_by -> {s_b}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
        if sort_value_counter > 1:
            raise ConfigError(f"Среди значение 'sort_by' только одно может иметь значение true в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Выводить"
        output_period = ['all_time', 'month', 'week', 'three_days', 'one_day']
        output_value_counter = 0
        for o_p in output_period:
            output_period_value = parameters['output_period'].get(o_p)
            output_value_counter += output_period_value
            if not isinstance(output_period_value, bool):
                raise ConfigError(f"Поле 'output_value -> {o_p}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
        if output_value_counter > 1:
            raise ConfigError(f"Среди значение 'output_period' только одно может иметь значение true в конфигурационном файле {config_yaml_path}")

        # Проверить все поля и значения настройки "Показывать на странице"
        output_size = ['show_20', 'show_50', 'show_100']
        output_size_value_counter = 0
        for o_s in output_size:
            output_size_value = parameters['output_size'].get(o_s)
            output_size_value_counter += output_size_value
            if not isinstance(output_size_value, bool):
                raise ConfigError(f"Поле 'output_size -> {o_s}' должно иметь тип bool в конфигурационном файле {config_yaml_path}")
        if output_size_value_counter > 1:
            raise ConfigError(f"Среди значение 'output_size' только одно может иметь значение true в конфигурационном файле {config_yaml_path}")
        
        return parameters
    
